# Projet Réseaux Bayésiens

# Importation des Données et Jointure 

In [37]:
import pandas as pd
import os

ot_odr_filename = os.path.join(".", "OT_ODR.csv.bz2")
ot_odr_df = pd.read_csv(ot_odr_filename,
                        compression="bz2",
                        sep=";")

equipements_filename = os.path.join(".", 'EQUIPEMENTS.csv')
equipements_df = pd.read_csv(equipements_filename,
                             sep=";")

# Fusion des DataFrames sur la colonne 'EQU_ID'
df_merged = pd.merge(ot_odr_df, equipements_df, on='EQU_ID')

# Affichage du DataFrame fusionné
df_merged

,OT_ID,ODR_ID,ODR_LIBELLE,TYPE_TRAVAIL,DUREE_TRAVAIL,SYSTEM_N1,SYSTEM_N2,SYSTEM_N3,EQU_ID,DATE_OT,KILOMETRAGE,SIG_ORGANE,SIG_CONTEXTE,SIG_OBS,LIGNE,MODELE,CONSTRUCTEUR,MOTEUR
0,OT000000000,OM000000000,REMPLACEMENT D'UNE GLACE LAT VOYAGEUR,CARROSSERIE,4.00,EQUIPEMENT DE CARROSSERIE,VITRAGE,VITRAGE LAT,E00005934,2011-03-29 19:26:06,149698.557783,GLACE/BAIE,INTERIEUR/GAUCHE/ARRIERE,DEBOITE,L0482,MD017,C007,MT014
1,OT000003071,OM000003274,REMPLACEMENT D'UN POTENTIOMETRE DE PORTE NUMERO 3,MECANIQUE,0.50,EQUIPEMENT DE CARROSSERIE,PORTE,SECURITES PORTES,E00005934,2012-09-29 21:30:27,212886.557303,PORTE,ARRIERE,FONCTIONNE MAL,L0135,MD017,C007,MT014
2,OT000003071,OM000003275,REMPLACEMENT D'UN POTENTIOMETRE DE PORTE NUMERO 4,MECANIQUE,0.50,EQUIPEMENT DE CARROSSERIE,PORTE,SECURITES PORTES,E00005934,2012-09-29 21:30:27,212886.557303,PORTE,ARRIERE,FONCTIONNE MAL,L0135,MD017,C007,MT014
3,OT000003071,OM000003276,REMPLACEMENT D'UN INTERRUPTEUR DE COMMANDE DE ...,MECANIQUE,0.08,EQUIPEMENT DE CARROSSERIE,PORTE,COMMANDE PORTE,E00005934,2012-09-29 21:30:27,212886.557303,PORTE,ARRIERE,FONCTIONNE MAL,L0135,MD017,C007,MT014
4,OT000005214,OM000005645,REMPLACEMENT DU CIRCUIT IMPRIME GAUCHE DU TABL...,MECANIQUE,0.20,EQUIPEMENT ELECTRIQUE,CIRCUIT DE GESTION ELECTRIQUE,CONNEXION - CABLAGE,E00005934,2013-02-11 21:45:40,228132.331835,DEMARRAGE,AU DEMARRAGE/POSTE CONDUITE,FAUX CONTACT,L0066,MD017,C007,MT014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506553,OT000403521,OM000503032,REMPLACEMENT D'UN FEU DE GABARIT,ELECTRICITE,0.04,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION EXT,E00364746,2019-09-17 00:22:38,1605.000000,ECLAIRAGE FEUX EXTERIEURS,LATERAL/DROIT,CASSE,L0098,MD064,C007,MT024
506554,OT000404355,OM000504846,REMPLACEMENT D'UN CABLAGE PASSE SANS CONTACT,ELECTRICITE,0.25,EQUIPEMENT EMBARQUE,TELEBILLETIQUE,PASSE SANS CONTACT,E00362393,2019-09-20 20:30:07,5145.078211,AVTT,GAUCHE,NE FONCTIONNE PAS,L0077,MD032,C006,MT016
506555,OT000405069,OM000505089,REMPLACEMENT D'UNE LAMPE DE CODE / PHARE,ELECTRICITE,0.04,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION EXT,E00364751,2019-09-24 23:31:41,9453.675111,ECLAIRAGE FEUX EXTERIEURS,GAUCHE/DROIT/AVANT,DESSERRE,L0098,MD064,C007,MT024
506556,OT000405447,OM000505789,REMPLACEMENT D'UN PASSE SANS CONTACT,EQUIPEMENT EMBARQUE,0.06,EQUIPEMENT EMBARQUE,TELEBILLETIQUE,PASSE SANS CONTACT,E00370753,2019-09-26 19:52:05,2599.051019,AVTT,EN MONTEE,VOYANT HS,L0178,MD021,C011,MT051


# One Hot Encoding - SIG_CONTEXTE

In [39]:
import pandas as pd

# Créer le DataFrame à partir des colonnes
df_contexte = pd.DataFrame(df_merged['SIG_CONTEXTE'])

# Diviser les valeurs de la colonne "SIG_CONTEXTE" en colonnes distinctes si elles n'existent pas déjà
if 'SIG_CONTEXTE' in df_contexte.columns:
    split_values = df_contexte["SIG_CONTEXTE"].str.split("/", expand=True)

# Appliquer l'encodage one-hot aux colonnes distinctes
one_hot_encoded = pd.get_dummies(split_values)

# Regrouper les colonnes encodées
grouped_encoded = one_hot_encoded.groupby(lambda x: x.split("_")[1], axis=1).sum()

# Ajouter un préfixe aux nouvelles colonnes
grouped_encoded = grouped_encoded.add_prefix("SIG_CONTEXTE=")

# Concaténer les colonnes encodées avec le DataFrame d'origine
df_contexte = pd.concat([df_contexte, grouped_encoded], axis=1)

# Afficher le DataFrame mis à jour
df_contexte

,SIG_CONTEXTE,SIG_CONTEXTE=A CHAUD,SIG_CONTEXTE=A FROID,SIG_CONTEXTE=A L'ACCELERATION,SIG_CONTEXTE=A L'ARRET,SIG_CONTEXTE=A L'OUVERTURE,SIG_CONTEXTE=A LA FERMETURE,SIG_CONTEXTE=A VIDE,SIG_CONTEXTE=ARRIERE,SIG_CONTEXTE=AU DEMARRAGE,...,SIG_CONTEXTE=HAUT,SIG_CONTEXTE=INTERIEUR,SIG_CONTEXTE=LATERAL,SIG_CONTEXTE=PLAFOND,SIG_CONTEXTE=PLATE FORME,SIG_CONTEXTE=PORTE,SIG_CONTEXTE=POSTE CONDUITE,SIG_CONTEXTE=REMORQUE,SIG_CONTEXTE=ROTONDE,SIG_CONTEXTE=TABLEAU DE BORD
0,INTERIEUR/GAUCHE/ARRIERE,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
1,ARRIERE,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,ARRIERE,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,ARRIERE,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,AU DEMARRAGE/POSTE CONDUITE,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506553,LATERAL/DROIT,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
506554,GAUCHE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
506555,GAUCHE/DROIT/AVANT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
506556,EN MONTEE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# One Hot Encoding - SIG_ORGANE

In [40]:
import pandas as pd

# Créer le DataFrame à partir des colonnes
df_organe = pd.DataFrame(df_merged['SIG_ORGANE'])

# Diviser les valeurs de la colonne "SIG_ORGANE" en colonnes distinctes si elles n'existent pas déjà
if 'SIG_ORGANE' in df_organe.columns:
    split_values = df_organe["SIG_ORGANE"].str.split("/", expand=True)

# Appliquer l'encodage one-hot aux colonnes distinctes
one_hot_encoded = pd.get_dummies(split_values)

# Regrouper les colonnes encodées
grouped_encoded = one_hot_encoded.groupby(lambda x: x.split("_")[1], axis=1).sum()

# Ajouter un préfixe aux nouvelles colonnes
grouped_encoded = grouped_encoded.add_prefix("SIG_ORGANE=")

# Concaténer les colonnes encodées avec le DataFrame d'origine
df_organe = pd.concat([df_organe, grouped_encoded], axis=1)

# Afficher le DataFrame mis à jour
df_organe

,SIG_ORGANE,SIG_ORGANE= DETRESSE,SIG_ORGANE= INTERRUPTEUR,SIG_ORGANE= POUSSOIR,SIG_ORGANE= TACHYGRAPHE,SIG_ORGANE=ABS,SIG_ORGANE=ACCELERATEUR ELECTRIQUE,SIG_ORGANE=ACCOUDOIR,SIG_ORGANE=AERATEURS DE TOITURE,SIG_ORGANE=AIR,...,SIG_ORGANE=VIDEO PORTE,SIG_ORGANE=VIDEO-SURVEILLANCE,SIG_ORGANE=VITESSE,SIG_ORGANE=VOITURE,SIG_ORGANE=VOLANT,SIG_ORGANE=VOYAGEUR,SIG_ORGANE=VOYANT ALARME ROUGE,SIG_ORGANE=VOYANT ALERTE ORANGE,SIG_ORGANE=VOYANT DE BATTERIE,SIG_ORGANE=VOYANT ITC
0,GLACE/BAIE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,PORTE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,PORTE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,PORTE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,DEMARRAGE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506553,ECLAIRAGE FEUX EXTERIEURS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
506554,AVTT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
506555,ECLAIRAGE FEUX EXTERIEURS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
506556,AVTT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
